# Проект SQL

Компания купила крупный сервис для чтения книг по подписке. В базе данных хранится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Наша задача как аналитика — проанализировать базу данных.

Для этого необходимо:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — для того, чтобы исключить из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывайть только книги с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
#sudo apt-get install build-dep python-psycopg2
#pip install psycopg2-binary

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

In [3]:
# формируем строку соединения с базой данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

In [4]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Описание данных

### Таблица books

In [5]:
query_books = '''SELECT * 
                 FROM books
                 LIMIT 1;
'''

In [6]:
books = pd.io.sql.read_sql(query_books, con = engine) 
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


Таблица `books` cодержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

### Таблица authors

In [7]:
query_authors = '''SELECT *
                   FROM authors
                   LIMIT 1;
'''

In [8]:
authors = pd.io.sql.read_sql(query_authors, con = engine) 
authors

,author_id,author
0,1,A.S. Byatt


Таблица `authors` cодержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

### Таблица publishers

In [9]:
query_publishers = '''SELECT *
                      FROM publishers
                      LIMIT 1;
'''

In [10]:
publishers = pd.io.sql.read_sql(query_publishers, con = engine) 
publishers

,publisher_id,publisher
0,1,Ace


Таблица `publishers` cодержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

### Таблица ratings

In [11]:
query_ratings = '''SELECT *
                   FROM ratings
                   LIMIT 1;
'''

In [12]:
ratings = pd.io.sql.read_sql(query_ratings, con = engine) 
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


Таблица `ratings` cодержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

### Таблица reviews

In [13]:
query_reviews = '''SELECT *
                   FROM reviews
                   LIMIT 1;
'''

In [14]:
reviews = pd.io.sql.read_sql(query_reviews, con = engine) 
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


Таблица `reviews` содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Задача 1

Посчитать, сколько книг вышло после 1 января 2000 года.

In [15]:
query_1 = '''SELECT COUNT(book_id) 
             FROM books
             WHERE CAST(publication_date as date) > '2000-01-01';
'''

In [16]:
query_1 = pd.io.sql.read_sql(query_1, con = engine) 
query_1

,count
0,819


**Вывод:** После 01 января 2000 года вышло 819 книг.

## Задача 2

Для каждой книги посчитать количество обзоров и среднюю оценку.

In [17]:
query_2 = '''SELECT b.book_id, b.title, COUNT(DISTINCT rev.review_id) as review_count, ROUND(AVG(rat.rating),1) as rating_avg
             FROM books as b
             LEFT JOIN reviews as rev ON b.book_id=rev.book_id
             LEFT JOIN ratings as rat ON b.book_id=rat.book_id
             GROUP BY b.book_id
             ORDER BY review_count DESC, rating_avg DESC;
'''

In [18]:
query_2 = pd.io.sql.read_sql(query_2, con = engine) 
query_2.head()

,book_id,title,review_count,rating_avg
0,948,Twilight (Twilight #1),7,3.7
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
2,656,The Book Thief,6,4.3
3,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.3
4,734,The Glass Castle,6,4.2



**Вывод:** Максимальное количество обзоров 7 на книгу 'Twilight(Twilight #1)' со средней оценкой 3.7

## Задача 3

Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — для того, чтобы исключить из анализа брошюры

In [19]:
query_3 = '''SELECT p.publisher, COUNT(b.book_id) as book_count
             FROM publishers as p
             JOIN books as b ON p.publisher_id=b.publisher_id
             WHERE b.num_pages > 50
             GROUP BY p.publisher_id
             ORDER BY book_count DESC
             LIMIT 1;
'''

In [20]:
query_3 = pd.io.sql.read_sql(query_3, con = engine) 
query_3

,publisher,book_count
0,Penguin Books,42


**Вывод:** Издательство 'Penguin Books' выпустило наибольшее число книг толще 50 страниц - 42шт

## Задача 4

Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.

In [21]:
query_4 = '''WITH max_rating AS (SELECT b.author_id, ROUND(AVG(r.rating),2) as avg_rating
                                 FROM books as b
                                 JOIN ratings as r ON b.book_id=r.book_id
                                 WHERE b.book_id IN (SELECT book_id
                                                     FROM ratings
                                                     GROUP BY book_id
                                                     HAVING COUNT(rating) >= 50)
                                 GROUP BY b.author_id
                                 ORDER BY avg_rating DESC
                                 LIMIT 1)
             
             SELECT a.author_id, a.author, mr.avg_rating
             FROM max_rating as mr
             JOIN authors as a ON mr.author_id=a.author_id;
'''

In [22]:
query_4 = pd.io.sql.read_sql(query_4, con = engine) 
query_4

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.29


**Вывод:** Автор с самой высокой средней оценкой книг 4.29 — J.K. Rowling/Mary GrandPré.

## Задача 5

Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [23]:
query_5 = ''' WITH count_review AS (SELECT username , COUNT(DISTINCT review_id) as count
                                    FROM reviews
                                    WHERE username IN (SELECT username
                                                       FROM ratings
                                                       GROUP BY username
                                                       HAVING COUNT(rating) >= 50)
                                    GROUP BY username)
                                    
              SELECT ROUND(AVG(count)) as avg_review  
              FROM count_review;
'''

In [24]:
query_5 = pd.io.sql.read_sql(query_5, con = engine) 
query_5

,avg_review
0,24.0


**Вывод:** Всреднем пользователи, которые поставили больше 50 оценок, делают 24 обзора.

<div class="alert alert-info">
    
Проанализировав базу данных, можно сказать что:
- С 1 января 2000 года вышло 819 книг.
- Наибольшее число книг, толще 50 страниц, выпустило издательство **Penguin Books** - 42шт,
- Книга **Twilight(Twilight #1)** получила максимальное количество обзоров - 7 и среднюю оценку - 3.7,
- **J.K. Rowling/Mary GrandPré** - автор с самой высокой средней оценкой книг 4.29,
- Пользователи, которые поставили больше 50 оценок, всреднем делают 24 обзора.
    
Эти данные помогут сформулировать ценностное предложение для нового продукта.    